In [1]:
! pip install hazm

In [5]:
import nltk

In [2]:
from __future__ import unicode_literals
from hazm import *


text = 'یک خودرو با طول دراز و عرض کوتاه از ما سبقت گرفت.'

tagger = POSTagger(model='resources/hazm/postagger.model')
tagger.tag(word_tokenize(text))

[('یک', 'NUM'),
 ('خودرو', 'N'),
 ('با', 'P'),
 ('طول', 'Ne'),
 ('دراز', 'AJ'),
 ('و', 'CONJ'),
 ('عرض', 'Ne'),
 ('کوتاه', 'AJ'),
 ('از', 'P'),
 ('ما', 'PRO'),
 ('سبقت', 'N'),
 ('گرفت', 'V'),
 ('.', 'PUNC')]

In [15]:
def tree_generator(sentence):
    grammar = r"""
        NADJ: {<N|Ne><ADV>*<AJ|AJe>}
    """
    return nltk.RegexpParser(grammar).parse(sentence)

noun_and_adj_list = []
for sentence in sent_tokenize('ماشین بسیار فوق‌العاده زیبا غذای خوب مداد علی شکم چاق'):
    tags = tagger.tag(word_tokenize(sentence))
    tree = tree_generator(tags)
    for subtree in tree.subtrees():
        # print(subtree.leaves())
        if subtree.label() == 'NADJ':
            noun_and_adj_list.append(subtree.leaves())
            print(subtree.leaves())  

[('ماشین', 'Ne'), ('بسیار', 'ADV'), ('فوق\u200cالعاده', 'ADV'), ('زیبا', 'AJ')]
[('غذای', 'Ne'), ('خوب', 'AJe')]
[('شکم', 'Ne'), ('چاق', 'AJ')]


In [23]:
p = Normalizer()
p.normalize('جابهجایی')

'جابهجایی'

In [8]:
chunker = Chunker(model='resources/hazm/chunker.model')
tagged = tagger.tag(word_tokenize(text))
a = chunker.parse(tagged)
b = tree2brackets(a)
b

'[یک خودرو NP] [با PP] [طول دراز و عرض کوتاه NP] [از PP] [ما NP] [سبقت گرفت VP] .'

In [7]:
c = a[0]
print(1)

NameError: name 'a' is not defined

In [26]:
d = ' '.join([s[0] for s in list(a[0])])

In [27]:
d

'سرعت بالای خودروی عمه علی'

In [29]:
list(a)

[Tree('NP', [('سرعت', 'Ne'), ('بالای', 'AJe'), ('خودروی', 'Ne'), ('عمه', 'N'), ('علی', 'N')]),
 Tree('ADJP', [('عجیب', 'AJ')]),
 Tree('VP', [('است', 'V')]),
 ('.', 'PUNC')]

In [5]:
from __future__ import unicode_literals
import hazm
import re
import json
import nltk


tagger = hazm.POSTagger(model='resources/hazm/postagger.model')
chunker = hazm.Chunker(model='resources/hazm/chunker.model')

In [24]:
inverse_alternative = {}
with open('resources/dataset/quantities_word_network.json', 'r', encoding='utf-8') as fp:
    inverse_alternative = json.load(fp)

In [59]:
def tree_generator(sentence):
    grammar = r"""
        NADJ: {<N|Ne><ADV>*<AJ|AJe|ADV>}
    """
    return nltk.RegexpParser(grammar).parse(sentence)

def extract_qualitative_expressions(text, tagger, inverse_alternative):
    results = []
    for sentence in hazm.sent_tokenize(text):
        tags = tagger.tag(hazm.word_tokenize(sentence))
        tree = tree_generator(tags)
        last_index = 0
        for subtree in tree.subtrees():
            if subtree.label() == 'NADJ':
                try:
                    in_context_POS = subtree.leaves()[-1][1]
                    out_of_context_POS = tagger.tag([subtree.leaves()[-1][0]])[0][1]
                    if in_context_POS.startswith("ADV") and out_of_context_POS.startswith("ADV"):
                        continue
                    try:
                        res_type = inverse_alternative[subtree.leaves()[0][0]]
                    except:
                        y_count = -1
                        while subtree.leaves()[0][0][y_count] == 'ی':
                            y_count -= 1
                        if y_count == -1:
                            continue
                        res_type = inverse_alternative[subtree.leaves()[0][0][:y_count + 1]]
                    marker = ' '.join([s[0] for s in subtree.leaves()])
                    span = re.search(marker, text[last_index:]).span()
                    span = [span[0] + last_index, span[1] + last_index]
                    last_index = span[1]
                    res_dict = {
                        'type': res_type,
                        'amount': '',
                        'unit': '',
                        'item': '',
                        'marker': marker,
                        'span': span,
                        'SI_equivalent': '',
                    }
                    results.append(res_dict)
                except KeyError:
                    pass
    return results

In [60]:
text = 'ما در ابتدا یک ماشین با سرعت زیاد دیدیم که در دمای بسیار سرد قطب جنوب در ارتفاع حقیقتا بسیار بلند در حال رانندگی بود. سپس مردی با قد کوتاه و شلوار خیلی بلند شروع به پیاده‌شدن کرد. قد آن مرد تقریباً برابر عرض حدودا متوسط آن ماشین بود. مردی با قد نسبتا کوتاه و پهنایی به‌غایت عریض'

extract_qualitative_expressions(text, tagger, inverse_alternative)

[{'type': 'سرعت',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'سرعت زیاد',
  'span': [24, 33],
  'SI_equivalent': ''},
 {'type': 'دما',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'دمای بسیار سرد',
  'span': [46, 60],
  'SI_equivalent': ''},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'ارتفاع حقیقتا بسیار بلند',
  'span': [73, 97],
  'SI_equivalent': ''},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'قد کوتاه',
  'span': [130, 138],
  'SI_equivalent': ''},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'قد نسبتا کوتاه',
  'span': [242, 256],
  'SI_equivalent': ''},
 {'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'پهنایی به\u200cغایت عریض',
  'span': [259, 278],
  'SI_equivalent': ''}]

In [12]:
inverse_alternative['پهنا']

'طول'

In [23]:
# res_dict = {
#     'type': inverse_alternative[NONE],
#     'amount': '',
#     'unit': '',
#     'item': '',
#     'marker': '',
#     'span': '',
# }

subtree.leaves()[0][0]

'شکم'

In [46]:
def join_patterns(patterns, grouping=False):
    return '(' + ('?:' if not grouping else '') + '|'.join(patterns) + ')'


def extract_qualitative(text, quantity, last_span):
    tagged = tagger.tag(word_tokenize(text))
    res = list(chunker.parse(tagged))
    for tree in res:
        tokens = list(tree)
        for idx in range(len(tokens)):
            if tokens[idx][0] == quantity and idx != len(tokens) - 1:
                chunk_str = ' '.join([s[0] for s in tokens])
                match_span = re.search(chunk_str, text).span()
                adj_count = 0
                while idx + adj_count + 1 < len(tokens) and tokens[idx + adj_count + 1][1].startswith('ADV'):
                    adj_count += 1
                target_token_pos = tokens[idx + adj_count + 1][1]
                if (target_token_pos.startswith('AJ') or target_token_pos.startswith('ADJ')) and match_span[0] > last_span:
                    return chunk_str

def qualitive_descriptions(text):
    sents = sent_tokenize(text)
    PATTERN = join_patterns(list(inverse_alternative.keys()))
    res = []
    read_chars = 0
    for sent in sents:
        last_span = -1
        for match in re.finditer(PATTERN, sent):
            marker = extract_qualitative(sent, match.group(), last_span)
            if marker is None:
                continue
            temp_dict = {
                'type': inverse_alternative[match.group()],
                'amount': '',
                'unit': '',
                'item': '',
                'marker': marker,
                'span': [match.span()[0] + read_chars, match.span()[0] + len(marker) + read_chars]
            }
            last_span = match.span()[0] + len(marker) - 1
            res.append(temp_dict)
        read_chars += len(sent)
        
    return res
        

In [47]:
p = Normalizer()

In [51]:
text = 'یک خودرو با طول دراز و عرض کوتاه از ما سبقت گرفت.'
text = 'طول بسیار زیبا و بسیار دراز خودروی علی شگفت‌انگیز است.'

b = 1 * text

b = p.normalize(b)

qualitive_descriptions(b)

[{'type': 'طول',
  'amount': '',
  'unit': '',
  'item': '',
  'marker': 'طول بسیار زیبا و بسیار دراز',
  'span': [0, 27]}]

In [24]:
import json
quantities = {}
with open('resources/dataset/quantities_related_names.json', 'r', encoding='utf-8') as fp:
    quantities = json.load(fp)

quantities['طول'].append('پهنا')

temp = {}

for k, v in quantities.items():
    for alt in v:
        if alt not in temp.keys():
            temp[alt] = k

with open('resources/dataset/inverse_alternative.json', 'w', encoding='utf-8') as fp:
    json.dump(temp, fp, indent=4)

In [22]:
a[0][0]

('سرعت', 'Ne')

In [26]:
import json
quantities = {}
with open('resources/dataset/quantities_related_names.json', 'r', encoding='utf-8') as fp:
    quantities = json.load(fp)

quantities_names = []
for k, v in quantities.items():
    quantities_names.extend(v)

quantities_names

with open('resources/dataset/alternatives_for_quantities.json', 'w', encoding='utf-8') as fp:
    json.dump(quantities_names, fp, indent=4)

In [27]:
alternatives = {}
with open('resources/dataset/alternatives_for_quantities.json', 'r', encoding='utf-8') as fp:
    alternatives = json.load(fp)

alternatives

['تغییرات سرعت به تغییرات زمان',
 'شتاب',
 'تغییرات سرعت نسبت به تغییرات زمان',
 'تغییرات سرعت نسبت به زمان',
 'گشتاور دورانی',
 'گشتاور زاویه\u200cای',
 'تکانه زاویه\u200cای',
 'سطح',
 'زمین',
 'مساحت',
 'چگالی سطح',
 'چگالی سطحی',
 'جرم سطحی',
 'ظرفیت',
 'خازن',
 'کاپاسیتانس',
 'پتانسیل شیمیایی',
 'آنتروپی',
 'آنتالپی',
 'چگالی جریان',
 'جریان الکتریکی بر واحد سطح',
 'بار الکتریکی',
 'بار',
 'الکتریسیته',
 'پتانسیل الکتریکی',
 'ولتاژ',
 'پتانسل',
 'اختلاف پتانسیل',
 'مقاومت الکتریکی',
 'مقاومت',
 'رزیستانس',
 'انرژی',
 'گرما',
 'ارزش غذایی',
 'کارمایه',
 'پتانسیل',
 'جنبشی',
 'آنتروپی',
 'آنتالپی',
 'توان',
 'نرخ',
 'فشار',
 'گشتاور',
 'نیروی چرخشی',
 'نیروی دورانی',
 'ممان نیرو',
 'سرعت',
 'تندی',
 'جابجایی نسبت به زمان',
 'مسافت نسبت به زمان',
 'جابجایی نسبت به تغییرات زمان',
 'مسافت نسبت به تغییرات زمان',
 'مسافت طی شده نسبت به تغییرات زمان',
 'حجم',
 'فضا',
 'طول',
 'فاصله',
 'عرض',
 'ارتفاع',
 'جابه\u200cجایی',
 'جا به جایی',
 'جابجایی',
 'مسافت',
 'بازه',
 'شعاع',
 'قطر',
 'ضلع

In [ ]:
class Extracted_quantity:

    def __init(self, type, amount, unit, item, marker, span, SI_equivalent):
        self.type = type
        self.amount = amount
        self.unit = unit
        self.item = item
        self.marker = marker
        self.span = span
        self.SI_equivalent = SI_equivalent

    def get_dict(self):
        return {
            'type': self.type,
            'amount': self.amount,
            'unit': self.unit,
            'item': self.item,
            'marker': self.marker,
            'span': self.span,
            'SI_equivalent': self.SI_equivalent,
        }

